In [1]:
import tensorflow as tf, numpy as np, pandas as pd, torch

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [3]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-w-landscape-full-25ms-24hr-28-2-24.feather')

df = df[df.f < df.fg]
df = df.dropna()
df['gust_factor'] = df.fg / df.f
df = df.drop(['f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'X', 'Y', 'time', 'stod'], axis = 1)

In [5]:
y = df.gust_factor
X = df.drop(['gust_factor'], axis = 1)

columns = X.columns

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)

scaler = StandardScaler()

# Assuming 'X' is your feature matrix and 'y' is your target variable
# Replace 'X' and 'y' with your actual data

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


train = pd.DataFrame(X_train, columns = columns)
train['gust_factor'] = y_train

test = pd.DataFrame(X_test, columns = columns)
test['gust_factor'] = y_test

In [6]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import NodeConfig, DANetConfig, GANDALFConfig 
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

data_config = DataConfig(
    target=["gust_factor"],
    continuous_cols=list(columns),  # List of continuous feature column names
    categorical_cols=[]#["cat_feature1", "cat_feature2", ...],  # List of categorical feature column names
)

model_config = GANDALFConfig(
    task="regression",  # or "classification" for classification tasks
    gflu_stages = 10,
    gflu_dropout = 0.2
)

optimizer_config = OptimizerConfig()

trainer_config = TrainerConfig(max_epochs=20)#, gradient_clip_val=10, fast_dev_run = False)

In [7]:
node_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2024-03-06 11:59:08,526 - {pytorch_tabular.tabular_model:140} - INFO -         
Experiment Tracking is turned off                                              


In [8]:
node_model.fit(train=train) #, validation=val)

Seed set to 42


2024-03-06 12:00:44,583 - {pytorch_tabular.tabular_model:524} - INFO -         
Preparing the DataLoaders                                                      
2024-03-06 12:00:44,588 - {pytorch_tabular.tabular_datamodule:499} - INFO -    
Setting up the datamodule for regression task                                  
2024-03-06 12:00:44,624 - {pytorch_tabular.tabular_model:574} - INFO -         
Preparing the Model: GANDALFModel                                              
2024-03-06 12:00:44,708 - {pytorch_tabular.models.gandalf.gandalf:109} - INFO -
Data Aware Initialization of T0                                                
2024-03-06 12:00:44,745 - {pytorch_tabular.tabular_model:340} - INFO -         
Preparing the Trainer                                                          


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


2024-03-06 12:00:45,043 - {pytorch_tabular.tabular_model:652} - INFO - Training
Started                                                                        
┌───┬──────────────────┬──────────────────┬────────┐
│   │ Name             │ Type             │ Params │
├───┼──────────────────┼──────────────────┼────────┤
│ 0 │ _backbone        │ GANDALFBackbone  │ 16.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer │     32 │
│ 2 │ _head            │ Sequential       │     18 │
│ 3 │ loss             │ MSELoss          │      0 │
└───┴──────────────────┴──────────────────┴────────┘
Trainable params: 16.1 K                                                       
Non-trainable params: 0                                                        
Total params: 16.1 K                                                           
Total estimated model params size (MB): 0                                      


Output()

c:\Users\Brynjar 
Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\
data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider 
increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:639: Checkpoint directory C:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\model\saved_models exists and is not empty.


c:\Users\Brynjar 
Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\
data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider 
increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

2024-03-06 12:01:51,724 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-03-06 12:01:51,727 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

In [9]:
result = node_model.evaluate(test)
y_pred = node_model.predict(test)

#tabular_model.save_model("examples/basic")
#loaded_model = TabularModel.load_model("examples/basic")

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.11775071918964386    │
│  test_mean_squared_error  │    0.11775071918964386    │
└───────────────────────────┴───────────────────────────┘

In [10]:
mape = mean_absolute_percentage_error(y_test, y_pred.to_numpy())
print(f"The MAPE is {mape}")

The MAPE is 10.462827682495117
